# Market Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
from eq.ds.qadirect.universes import sp_500
from eq.ds.qadirect.selectables import trds_closing_prices, trds_adj_factors, trds_dividends, trds_market_value
from eq.lqr.lib.bbg.bpipe_refdata import BloombergRefDataService

### Get Trading Dates
Given the start date and the end date, return all the market-open dates within. Take INTC (infocode=46712) as the fiducial.

In [3]:
def get_trading_dates(start_date, end_date):
    return trds_closing_prices('46712', startdate=start_date, enddate=end_date)[["MarketDate"]]

### Get Rebalance Dates
Rebalance on the sameday every week:

In [4]:
def get_rebalance_dates(trading_dates, interval = 7):
    start_date, end_date = trading_dates["MarketDate"].iloc[0], trading_dates["MarketDate"].iloc[-1]
    day = start_date
    ix = []
    while day <= end_date:
        days = trading_dates[trading_dates["MarketDate"] >= day]
        if(len(days)) > 0:
            ix.append(days.index[0])
        day += datetime.timedelta(interval)
    rebalance_dates = trading_dates["MarketDate"].iloc[ix]
    rebalance_dates.index = range(len(rebalance_dates))
    return rebalance_dates

### Get Closing Prices
Given the infocode (can be found from the "sp_500" function in "eq.ds.qadirect.universes") and the trading dates, return the closing price time sereis as a pandas Series.

In [5]:
def get_closing_prices(infocode, trading_dates):
    start_date, end_date = trading_dates["MarketDate"].iloc[0], trading_dates["MarketDate"].iloc[-1]
    raw_prices = trds_closing_prices(infocode, startdate = start_date, enddate=end_date)
    split_adjustments = trds_adj_factors(infocode, startdate = start_date, enddate=end_date)
    dividends = trds_dividends(infocode, startdate = start_date, enddate=end_date)
    raw_prices = pd.merge(trading_dates, raw_prices, on="MarketDate", how="left")
    raw_prices.fillna(method="ffill", inplace=True)
    #filtered_prices.dropna(inplace=True)
    adjusted_prices = raw_prices.copy()
    if split_adjustments is not None:
        for i in range(0, len(split_adjustments)):
            adj_factor = split_adjustments.iloc[i]["AdjFactor"]
            if (adj_factor is not None) and (not np.isnan(adj_factor)):
                adjusted_prices.loc[adjusted_prices["MarketDate"] < split_adjustments.iloc[i]["AdjDate"], ['PX']] *= adj_factor
    if dividends is not None:
        for i in range(0, len(dividends)):
            adjusted_prices.loc[adjusted_prices["MarketDate"] < dividends.iloc[i]["EffectiveDate"], ['PX']] *= (1 - dividends.iloc[i]["Div"] / raw_prices.iloc[raw_prices[raw_prices["MarketDate"] >= dividends.iloc[i]["EffectiveDate"]].index[0]-1]['PX'])
    return pd.Series(adjusted_prices["PX"].values, index=adjusted_prices["MarketDate"].values)

### Get Rolling Investment Universe
We trade on each rebalance date. 

This function returns the investment universe on each rebalance date (as an array of infocode), as well as an infocode dictionary, which map infocode to ticker name.

In [6]:
def get_rolling_universe(trading_dates):
    rolling_universe = {}
    infocode_dict = {}
    infocode_to_trid = {}
    for date in trading_dates["MarketDate"]:
        data = sp_500(asofdate=date)
        rolling_universe[date] = np.array(data["INFOCODE"])
        print(date, len(rolling_universe[date]))
        infocode_dict.update(dict(pd.Series(data["TICKER"].values, index=data["INFOCODE"]).astype("string")))
        infocode_to_trid.update(dict(pd.Series(data["ID"].values, index=data["INFOCODE"]).astype("string")))
    return rolling_universe, infocode_dict, infocode_to_trid

### Remov Invalid Names from Investment Universe
Given a list of invalid infocodes to be excluded, remove those names in the rolling and total universe inplace.

In [7]:
def remove_from_universe(rolling_universe, infocode_dict, infocode_to_trid, exclude_infocode):
    new_rolling_universe, new_infocode_dict, new_infocode_to_trid = {}, {}, {}
    for key in rolling_universe:
        new_rolling_universe[key] = np.array(list(set(rolling_universe[key]) - exclude_infocode))  
    new_infocode_dict =  pd.Series(infocode_dict)[set(infocode_dict.keys()) - exclude_infocode].to_dict()
    new_infocode_to_trid =  pd.Series(infocode_to_trid)[set(infocode_to_trid.keys()) - exclude_infocode].to_dict()
    return new_rolling_universe, new_infocode_dict, new_infocode_to_trid

### Get Ticker Dictionary
Given the infocode dictionary, returns the ticker dictionary, which maps ticker name to infocode.

In [8]:
def get_ticker_dict(info_dict):
    ticker_dict = pd.Series(infocode_dict.keys(), index=infocode_dict.values()).to_dict()
    return ticker_dict

### Get Stock Price Time Series
This function returns a dictionary which map each infocode in the investment universe to a closing price time series. The range of the timeseries is determined by the trading dates.

In [9]:
def get_stock_prices(infocode_dict, trading_dates):
    stock_prices = pd.DataFrame(index=trading_dates["MarketDate"])
    for infocode in infocode_dict:
        #print(infocode)
        stock_prices[infocode] = get_closing_prices(infocode, trading_dates)
    return stock_prices

### Get Stock Shares Outstanding and Market Capitalization Time Series

In [10]:
def get_stock_shares(infocode_dict, trading_dates):
    start_date, end_date = trading_dates["MarketDate"].iloc[0], trading_dates["MarketDate"].iloc[-1]
    stock_shares = pd.DataFrame(index=trading_dates["MarketDate"])
    for infocode in infocode_dict:
        #print(infocode)
        data = trds_market_value(infocode, start_date, end_date)
        if data is not None:
            stock_shares[infocode] = pd.Series(data['ConsolNumShrs'].values, index=data['ValDate'])
        else:
            stock_shares[infocode] = pd.Series()
    stock_shares.fillna(method='ffill', inplace=True)
    return stock_shares

In [11]:
def get_stock_marketcap(infocode_dict, trading_dates):
    start_date, end_date = trading_dates["MarketDate"].iloc[0], trading_dates["MarketDate"].iloc[-1]
    stock_marketcap = pd.DataFrame(index=trading_dates["MarketDate"])
    for infocode in infocode_dict:
        #print(infocode)
        data = trds_market_value(infocode, start_date, end_date)
        if data is not None:
            stock_marketcap[infocode] = pd.Series(data['ConsolMktVal'].values, index=data['ValDate'])
        else:
            stock_marketcap[infocode] = pd.Series()
    stock_marketcap.fillna(method='ffill', inplace=True)
    return stock_marketcap

### Get Stock Implied Volatilities
Given Bloomberg tickers and a list of trading dates, return stock implied volatilities as a dict with ticker as the key.

In [12]:
def get_stock_vols(infocode_dict, trading_dates, field='3MO_CALL_IMP_VOL'):
    start_date, end_date = trading_dates["MarketDate"].iloc[0], trading_dates["MarketDate"].iloc[-1]
    # Convert to a list of BBG ticker names
    tickers = infocode_dict.values()
    for i in range(len(tickers)):
        tickers[i] += ' US Equity'
        if '.' in tickers[i]:
            tickers[i] = tickers[i].replace('.', '/')
    # Query IV data from BBG
    refDataService = BloombergRefDataService()
    result, _, _ = refDataService.makeHistoricalDataRequest( tickers, [field], start_date, end_date )
    stock_vols = pd.DataFrame(index=trading_dates["MarketDate"])
    for ticker in tickers:
        infocode = ticker_dict[ticker.split()[0].replace('/', '.')]
        stock_vols[infocode] = pd.Series(result[ticker][field]) / 100
    #stock_vols.fillna(method="ffill", inplace=True)
    return stock_vols

# Data Loading Pipeline

In [57]:
start_date = datetime.date(2005,1,1)
end_date = datetime.date(2018,6,1)
#exclude_infocode = set([71734, 61565, 64508, 60749, 65072 ,51032 ,65373, 294908, 63959, 70369, 54356, 63713, 67195, 42992, 293511, 306512, 153450])
#exclude_infocode = set([ 63610,  61565, 255410,  64265,  51995,  64296,  64508,  60749, 65072,  71734,  51032,  65373,  57296, 294908])
#exclude_infocode = set([ 63610,  61565, 255410,  64265,  51995,  64296,  64508,  60749, 65072,  71734,  51032,  65373,  57296, 294908, 49347, 51438, 261769, 267369])
exclude_infocode = set([ 57418,  63610,  61565,  50890,  49347,  51438, 145705, 123181,
        71734, 160169, 255410, 293511,  64265,  51995,  64296,  62390,
        64508, 261769,  62473, 267369, 216209, 263351, 249020,  60749,
        60923,  65072, 270062, 249641,  51032,  65373,  57296, 294908])

In [59]:
trading_dates = get_trading_dates(start_date, end_date)
trading_dates.head()

In [60]:
trading_dates.count()

In [19]:
rolling_universe, infocode_dict, infocode_to_trid = get_rolling_universe(trading_dates)

In [61]:
rolling_universe[trading_dates["MarketDate"][0]][0:10]

In [62]:
len(infocode_dict)

In [63]:
len(infocode_to_trid)

In [64]:
infocode_dict[72990]

In [65]:
rolling_universe, infocode_dict, infocode_to_trid = remove_from_universe(rolling_universe, infocode_dict, infocode_to_trid, exclude_infocode)
print(len(infocode_dict))

In [66]:
ticker_dict = get_ticker_dict(infocode_dict)
print(len(ticker_dict))

In [70]:
print(len(infocode_dict), len(ticker_dict), len(infocode_to_trid))

In [69]:
ticker_dict['AAPL']

In [68]:
print(len(infocode_dict), len(ticker_dict), len(infocode_to_trid))

In [ ]:
stock_prices = get_stock_prices(infocode_dict, trading_dates)
stock_prices.head()

In [71]:
stock_shares = get_stock_shares(infocode_dict, trading_dates)
stock_shares.head()

In [72]:
stock_marketcap = get_stock_marketcap(infocode_dict, trading_dates)
stock_marketcap.head()

In [73]:
stock_iv3m = get_stock_vols(infocode_dict, trading_dates, field='3MO_CALL_IMP_VOL')
stock_iv3m.head()

In [74]:
stock_iv6m = get_stock_vols(infocode_dict, trading_dates, field='6MO_CALL_IMP_VOL')
stock_iv6m.head()

In [75]:
stock_iv12m = get_stock_vols(infocode_dict, trading_dates, field='12MO_CALL_IMP_VOL')
stock_iv12m.head()

In [96]:
stock_iv2m = get_stock_vols(infocode_dict, trading_dates, field='CALL_IMP_VOL_60D')
stock_iv2m.head()

In [95]:
stock_iv1m = get_stock_vols(infocode_dict, trading_dates, field='CALL_IMP_VOL_30D')
stock_iv1m.head()

In [97]:
stock_iv10d = get_stock_vols(infocode_dict, trading_dates, field='CALL_IMP_VOL_10D')
stock_iv10d.head()

In [76]:
stock_iv1m50delta = get_stock_vols(infocode_dict, trading_dates, field='1M_CALL_IMP_VOL_50DELTA_DFLT')
stock_iv1m50delta.head()

In [77]:
stock_iv1m25delta = get_stock_vols(infocode_dict, trading_dates, field='1M_CALL_IMP_VOL_25DELTA_DFLT')
stock_iv1m25delta.head()

In [78]:
stock_iv1m75delta = get_stock_vols(infocode_dict, trading_dates, field='1M_CALL_IMP_VOL_75DELTA_DFLT')
stock_iv1m75delta.head()

In [79]:
stock_iv2m50delta = get_stock_vols(infocode_dict, trading_dates, field='2M_CALL_IMP_VOL_50DELTA_DFLT')
stock_iv2m50delta.head()

In [80]:
stock_iv2m25delta = get_stock_vols(infocode_dict, trading_dates, field='2M_CALL_IMP_VOL_25DELTA_DFLT')
stock_iv2m25delta.head()

In [81]:
stock_iv2m75delta = get_stock_vols(infocode_dict, trading_dates, field='2M_CALL_IMP_VOL_75DELTA_DFLT')
stock_iv2m75delta.head()

# Save Loaded Data

In [82]:
directory = "h:/test_2005/"

In [83]:
trading_dates.to_csv(directory + "trading_dates.csv")

In [84]:
dic = {}
for key in rolling_universe:
    dic[key] = " ".join(str(x) for x in rolling_universe[key])
dic = pd.Series(dic)
pd.DataFrame(dic.values, index=dic.index, columns=['Infocode']).to_csv(directory+"rolling_universe.csv")

In [85]:
pd.DataFrame(pd.Series(infocode_dict).values, index=pd.Series(infocode_dict).index, columns=['Ticker']).to_csv(directory + "infocode_dict.csv")

In [86]:
pd.DataFrame(pd.Series(infocode_to_trid).values, index=pd.Series(infocode_to_trid).index, columns=['ID']).to_csv(directory + "infocode_to_trid.csv")

In [87]:
pd.DataFrame(pd.Series(ticker_dict).values, index=pd.Series(ticker_dict).index, columns=['Infocode']).to_csv(directory + "ticker_dict.csv")

In [88]:
stock_prices.to_csv(directory + "stock_prices.csv")
stock_shares.to_csv(directory + "stock_shares.csv")
stock_marketcap.to_csv(directory + "stock_marketcap.csv")

In [89]:
stock_iv3m.to_csv(directory + "stock_iv3m.csv")
stock_iv6m.to_csv(directory + "stock_iv6m.csv")
stock_iv12m.to_csv(directory + "stock_iv12m.csv")

In [98]:
stock_iv2m.to_csv(directory + "stock_iv2m.csv")
stock_iv1m.to_csv(directory + "stock_iv1m.csv")
stock_iv10d.to_csv(directory + "stock_iv10d.csv")

In [90]:
stock_iv1m50delta.to_csv(directory + "stock_iv1m50delta.csv")
stock_iv1m25delta.to_csv(directory + "stock_iv1m25delta.csv")
stock_iv1m75delta.to_csv(directory + "stock_iv1m75delta.csv")

In [91]:
stock_iv2m50delta.to_csv(directory + "stock_iv2m50delta.csv")
stock_iv2m25delta.to_csv(directory + "stock_iv2m25delta.csv")
stock_iv2m75delta.to_csv(directory + "stock_iv2m75delta.csv")

# Read Saved Data

In [17]:
trading_dates = pd.read_csv(directory + "trading_dates.csv", index_col=0)
trading_dates["MarketDate"] = pd.to_datetime(trading_dates["MarketDate"])

In [18]:
data = pd.read_csv(directory+"rolling_universe.csv", index_col=0)["Infocode"]
data.index = pd.to_datetime(data.index)
rolling_universe = {}
for key in data.index:
    rolling_universe[key] = np.array(data[key].split()).astype('int')

In [19]:
infocode_dict = pd.read_csv(directory + "infocode_dict.csv", index_col=0)["Ticker"].to_dict()

In [333]:
infocode_to_trid = pd.read_csv(directory + "infocode_to_trid.csv", index_col=0)["ID"].to_dict()

In [20]:
ticker_dict = pd.read_csv(directory + "ticker_dict.csv", index_col=0)["Infocode"].to_dict()

In [445]:
stock_prices = pd.read_csv(directory + "stock_prices.csv", index_col=0)
stock_prices.index = pd.to_datetime(stock_prices.index)
stock_prices.columns = stock_prices.columns.astype("int")

stock_shares = pd.read_csv(directory + "stock_shares.csv", index_col=0)
stock_shares.index = pd.to_datetime(stock_shares.index)
stock_shares.columns = stock_shares.columns.astype("int")

stock_marketcap = pd.read_csv(directory + "stock_marketcap.csv", index_col=0)
stock_marketcap.index = pd.to_datetime(stock_marketcap.index)
stock_marketcap.columns = stock_marketcap.columns.astype("int")

In [85]:
stock_iv3m = pd.read_csv(directory + "stock_iv3m.csv", index_col=0)
stock_iv3m.index = pd.to_datetime(stock_iv3m.index)
stock_iv3m.columns = stock_iv3m.columns.astype("int")

stock_iv6m = pd.read_csv(directory + "stock_iv6m.csv", index_col=0)
stock_iv6m.index = pd.to_datetime(stock_iv6m.index)
stock_iv6m.columns = stock_iv6m.columns.astype("int")

stock_iv12m = pd.read_csv(directory + "stock_iv12m.csv", index_col=0)
stock_iv12m.index = pd.to_datetime(stock_iv12m.index)
stock_iv12m.columns = stock_iv12m.columns.astype("int")

In [ ]:
stock_iv1m50delta = pd.read_csv(directory + "stock_iv1m50delta.csv", index_col=0)
stock_iv1m50delta.index = pd.to_datetime(stock_iv1m50delta.index)
stock_iv1m50delta.columns = stock_iv1m50delta.columns.astype("int")

stock_iv1m25delta = pd.read_csv(directory + "stock_iv1m25delta.csv", index_col=0)
stock_iv1m25delta.index = pd.to_datetime(stock_iv1m25delta.index)
stock_iv1m25delta.columns = stock_iv1m25delta.columns.astype("int")

stock_iv1m75delta = pd.read_csv(directory + "stock_iv1m75delta.csv", index_col=0)
stock_iv1m75delta.index = pd.to_datetime(stock_iv1m75delta.index)
stock_iv1m75delta.columns = stock_iv1m75delta.columns.astype("int")

# Test

In [29]:
len(set(infocode_to_trid.keys()))

In [30]:
len(set(infocode_to_trid.values()))

In [31]:
len(set(infocode_dict.keys()))

In [32]:
len(set(infocode_dict.values()))

In [34]:
len(set(ticker_dict.keys()))

In [35]:
set(infocode_dict.keys()) - set(ticker_dict.values())

In [50]:
repeated_tickers = []
for infocode in set(infocode_dict.keys()) - set(ticker_dict.values()):
    repeated_tickers.append(infocode_dict[infocode])
repeated_tickers

In [51]:
trid_to_infocode = {}
for infocode in infocode_to_trid:
    trid_to_infocode[infocode_to_trid[infocode]] = infocode

repeated_trids = []
for infocode in set(infocode_to_trid.keys()) - set(trid_to_infocode.values()):\
    repeated_trids.append(infocode_to_trid[infocode])
repeated_trids

In [53]:
exclude_infocode = []
for infocode in infocode_dict:
    if (infocode_dict[infocode] in repeated_tickers) or (infocode_to_trid[infocode] in repeated_trids):
        exclude_infocode.append(infocode)

In [58]:
len(exclude_infocode)

In [56]:
np.array(exclude_infocode)